In [25]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from pipeline_utils import (
    COMPUTE, CONCURRENT_TRIALS, DELAY_EVALUATION, EVALUATION_INTERVAL, GOAL, METRIC, SAMPLING_ALGORITHM, TIMEOUT, TIMEOUT_PLUS, TOTAL_TRIALS, #SWEEP
    SUBSCRIPTION, RESOURCE_GROUP, WS_NAME,  #AUTHENTICATE
    GBC_PATH, NBC_PATH, RFC_PATH, SVC_PATH, XGB_PATH, MULT_NBC_PATH,    #COMPONENTS PATHS
    GINI_PATH,    #FEAT SEL PATHS
    PREP_DATA_PATH,  #PREP DATA PATH
    GBC_BY_GINI, NBC_BY_GINI, RFC_BY_GINI, SVC_BY_GINI, XGB_BY_GINI, MULT_NBC_BY_GINI,   #PIPELINES
    GINI,   #FEAT SEL
    N_FEATURES, TRAIN_DATA, TEST_DATA, DATA_VERSION, #PIPELINE VALUES
    get_experiment_names,   #FUNCTIONS
)

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [26]:
data_to_train = ml_client.data.get(name=TRAIN_DATA.split('.')[0], version=DATA_VERSION)
print(f"Data to train asset URI: {data_to_train.path} - name: {TRAIN_DATA.split('.')[0]}")

data_to_test = ml_client.data.get(name=TEST_DATA.split('.')[0], version=DATA_VERSION)
print(f"Data to test asset URI: {data_to_test.path} - name: {TEST_DATA.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests/datastores/workspaceblobstore/paths/LocalUpload/3cfdebd258ea2db7363c55bb841ca887/vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv - name: vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017
Data to test asset URI: azureml://subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests/datastores/workspaceblobstore/paths/LocalUpload/3992df39ce41e34367e629d6656f4e9f/vrex_encoded_tf_idf_2018_2019_2020_2021.csv - name: vrex_encoded_tf_idf_2018_2019_2020_2021


In [27]:
# importing the Component Package
from azure.ai.ml import load_component

data_prep_component = load_component(source=PREP_DATA_PATH)
data_prep_component = ml_client.create_or_update(data_prep_component)

feat_sel_component = load_component(source=GINI_PATH)
feat_sel_component = ml_client.create_or_update(feat_sel_component)

train_gbc = load_component(source=GBC_PATH)
train_gbc = ml_client.create_or_update(train_gbc)

train_nbc = load_component(source=NBC_PATH)
train_nbc = ml_client.create_or_update(train_nbc)

train_rfc = load_component(source=RFC_PATH)
train_rfc = ml_client.create_or_update(train_rfc)

train_svc = load_component(source=SVC_PATH)
train_svc = ml_client.create_or_update(train_svc)

train_xgb = load_component(source=XGB_PATH)
train_xgb = ml_client.create_or_update(train_xgb)

mult_nbc_train = load_component(source=MULT_NBC_PATH)
mult_nbc_train = ml_client.create_or_update(mult_nbc_train)


In [28]:

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

In [29]:
@dsl.pipeline(
    name=GBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_gbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_gbc_job = train_gbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        n_estimators_to_gbc=Choice(values=[50, 100, 200]),
        learning_rate_to_gbc=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_gbc = train_gbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_gbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)

    sweep_step_to_gbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [30]:
@dsl.pipeline(
    name=NBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_nbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_nbc_job = train_nbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
    )

In [31]:
@dsl.pipeline(
    name=RFC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_rfc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_rfc_job = train_rfc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel, 
        n_estimators_to_rfc=Choice(values=[50, 100, 200]),
    )

    sweep_step_to_rfc = train_rfc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_rfc.set_limits(max_total_trials=TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_rfc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [32]:
@dsl.pipeline(
    name=SVC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_svc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_svc_job = train_svc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        kernel_to_svc=Choice(values=["linear", "rbf", "poly", "sigmoid","precomputed"]),
        gamma_to_svc=Choice(values=["scale", "auto"]),
    )

    sweep_step_to_svc = train_svc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_svc.set_limits(max_total_trials=4*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=2*TIMEOUT_PLUS)
    sweep_step_to_svc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [33]:
@dsl.pipeline(
    name=XGB_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_xgb_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_xgb_job = train_xgb(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        n_estimators_to_xgb=Choice(values=[100, 500, 1000]),
        learning_rate_to_xgb=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_xgb = train_xgb_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_xgb.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_xgb.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [34]:
@dsl.pipeline(
    name=MULT_NBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_mult_nbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    mult_nbc_train_job = mult_nbc_train(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        alpha=Choice(values=[0.01, 0.1, 0.5, 1.0, 2, 5]),
        fit_prior=Choice(values=[True, False]),
    )

    sweep_step_to_mult_nbc = mult_nbc_train_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_mult_nbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)

    sweep_step_to_mult_nbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [35]:
pipelines = []

for n_feature in N_FEATURES:
    pipelines.append(train_mult_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_gbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_rfc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))
    
    pipelines.append(train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_xgb_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))


In [36]:
experiment_names = get_experiment_names(GINI)
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )
    ml_client.jobs.stream(pipeline_job.name)

Gini_MultGaussianNB_N_FEAT_399_2024_06_23_20_00_54
Gini_GaussianNB_N_FEAT_399_2024_06_23_20_00_54
Gini_GradientBoostingClassifier_N_FEAT_399_2024_06_23_20_00_54
Gini_RandomForestClassifier_N_FEAT_399_2024_06_23_20_00_54
Gini_SVC_N_FEAT_399_2024_06_23_20_00_54
Gini_XGBoost_N_FEAT_399_2024_06_23_20_00_54
Gini_MultGaussianNB_N_FEAT_300_2024_06_23_20_00_54
Gini_GaussianNB_N_FEAT_300_2024_06_23_20_00_54
Gini_GradientBoostingClassifier_N_FEAT_300_2024_06_23_20_00_54
Gini_RandomForestClassifier_N_FEAT_300_2024_06_23_20_00_54
Gini_SVC_N_FEAT_300_2024_06_23_20_00_54
Gini_XGBoost_N_FEAT_300_2024_06_23_20_00_54
Gini_MultGaussianNB_N_FEAT_200_2024_06_23_20_00_54
Gini_GaussianNB_N_FEAT_200_2024_06_23_20_00_54
Gini_GradientBoostingClassifier_N_FEAT_200_2024_06_23_20_00_54
Gini_RandomForestClassifier_N_FEAT_200_2024_06_23_20_00_54
Gini_SVC_N_FEAT_200_2024_06_23_20_00_54
Gini_XGBoost_N_FEAT_200_2024_06_23_20_00_54
Gini_MultGaussianNB_N_FEAT_150_2024_06_23_20_00_54
Gini_GaussianNB_N_FEAT_150_2024_06_23

RunId: maroon_tiger_hyjhcq2sdk
Web View: https://ml.azure.com/runs/maroon_tiger_hyjhcq2sdk?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

Streaming logs/azureml/executionlogs.txt

[2024-06-23 20:01:00Z] Submitting 1 runs, first five are: d6ed2403:91d8aede-d328-4f1a-a6a1-9d425f19e237
[2024-06-23 20:01:01Z] Completing processing run id 91d8aede-d328-4f1a-a6a1-9d425f19e237.
[2024-06-23 20:01:02Z] Submitting 1 runs, first five are: b8feaccf:f8d8d1c1-5bfa-4d6f-81a1-2c129c5e4486
[2024-06-23 20:05:06Z] Completing processing run id f8d8d1c1-5bfa-4d6f-81a1-2c129c5e4486.
[2024-06-23 20:05:07Z] Submitting 1 runs, first five are: 68d3e3cc:41f178af-6d03-4e4a-b61a-5f2f2d0dc174
[2024-06-23 20:16:56Z] Completing processing run id 41f178af-6d03-4e4a-b61a-5f2f2d0dc174.

Execution Summary
RunId: maroon_tiger_hyjhcq2sdk
Web View: https://ml.azure.com/runs/maroon_tiger_hyjhcq2sdk?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resou

Bad pipe message: %s [b'\xbf\xfcadI\xe6.x\xce7N\xe33\x0bV1-h \xbcF\xf4\x9aJ\xcc6\xb3\xf2\xfc=C_E{\xbd\x1b\\J,\x8e*\x8f;.\x84\x81\xa6\x1e\xe5\xaf%\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xc3\x93o\xa5O\x0cr\xcbo\xa0V"\xb5%\xf5\xf5\xef\xf7\xa9\x8c\x07\x1f\xc3\xa6\xed0']
Bad pipe message: %s [b'\x8b5\x14\xb6\xc6\xa0,~\x9c\xe4$mJWV\xc9\xcf\xe8 \xd5\xd3\xc3v\xc0\x8a', b'\n\xe0\n', b'\x02P\\\x80\xab\xa4\xd9\xa0lz\x86\xc4\xc2\xf2\xec\xc6#Z\xbe\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x

[2024-06-24 01:13:00Z] Experiment cancel requested, checking remained jobs...
[2024-06-24 01:13:01Z] Experiment status is canceled, set experiment status as canceled and return.

Execution Summary
RunId: polite_kitten_q4nz7vl7pm
Web View: https://ml.azure.com/runs/polite_kitten_q4nz7vl7pm?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

Warnings:
Pipeline job canceled because its child job [name: sweep_step_to_svc, id: 04daffae-d2c1-4ef0-a282-5da63e0fc6d5] was canceled.


RunId: witty_honey_8jrgp3g59y
Web View: https://ml.azure.com/runs/witty_honey_8jrgp3g59y?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

Streaming logs/azureml/executionlogs.txt

[2024-06-24 01:13:20Z] Completing processing run id d14a0116-59bb-48aa-b03a-96cf594004fe.
[2024-06-24 01:13:21Z] Completing processing run id d2f8588d-1bae-4705-a67f-571b6028bfcc.
[2024-06-24 01:13:21Z] Submitting 1

Bad pipe message: %s [b'L\xcf\xd8D\xac\x85B\xfd&*\x00=\x98\xeeX\x0b$z s\r\x18\xa9<x\x1d\x81\x0e\xf3\xa8"\xb1\xb5\xcd\xe2\xf1\xbc\xba\xbeW~~\xb3\xbe\x0c\x84\x06}\x05\x9dk\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c', b'\x05\x03\x06', b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 ~0s\x89\xbe\xa8\x8e]\x80\xd0n7\xa2?\xd4{y\xe5\x0c\xa9z#']
Bad pipe message: %s [b"\x8fI\x04$\xf3\xd7\x8e\x16\x8b\xa3y\tn\x0e\x1e\x8fKh\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\

[2024-06-24 09:59:46Z] Completing processing run id ef25c18d-2147-483b-83df-2af6937168e7.

Execution Summary
RunId: affable_parang_0gkswjtnzz
Web View: https://ml.azure.com/runs/affable_parang_0gkswjtnzz?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

RunId: busy_bell_ws9z849ts8
Web View: https://ml.azure.com/runs/busy_bell_ws9z849ts8?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

Streaming logs/azureml/executionlogs.txt

[2024-06-24 10:00:24Z] Completing processing run id fcaf98bf-8dca-4625-bf57-7c715be6d8b1.
[2024-06-24 10:00:25Z] Completing processing run id a172c268-10e4-4e07-94fd-4c6407198b24.
[2024-06-24 10:00:26Z] Submitting 1 runs, first five are: 0f52769c:7f9ceb45-ee98-4f9e-b56c-56fbb05bbe24
[2024-06-24 10:26:47Z] Completing processing run id 7f9ceb45-ee98-4f9e-b56c-56fbb05bbe24.

Execution Summary
RunId: busy_bell_ws9z849ts8
Web View: https://ml.a

Bad pipe message: %s [b"U\x19\xf0\x0b\xb8\xe3M\xe3\xe0\x95\x07\x1b?\x00\x14\x80o^ \x89!\x98\xdc\xb5\x8d\x81\xb6\\W\xac8\x19\x00\xf2'P\x88s.OS\xed\xb0\x85t\xb9\xd2Ac\xca\xb9\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00", b'\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06']
Bad pipe message: %s [b'\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\x08\x06\x04\x01\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x14M\xa2\xb4U\xb3\xb7\xb3%\x8d\xaaT\xa2bpt\xbe\xc0\x82_\n2']
Bad pipe message: %s [b'\xa7(\xd4jw\xe8\\\xda\x1e\x07\xcd\xbf6\xd8a\xcd\xe0y\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00', b"\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\x

[2024-06-24 22:56:01Z] Completing processing run id e748cda5-13c0-4d6d-ab2e-87f520265873.

Execution Summary
RunId: tidy_parrot_lx0zvkh849
Web View: https://ml.azure.com/runs/tidy_parrot_lx0zvkh849?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

RunId: blue_music_r6b0v9m0t2
Web View: https://ml.azure.com/runs/blue_music_r6b0v9m0t2?wsid=/subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests

Streaming logs/azureml/executionlogs.txt

[2024-06-24 22:56:30Z] Completing processing run id 7ce08d89-aad6-4dc0-a6d4-7c810d74c13a.
[2024-06-24 22:56:31Z] Completing processing run id 2fc321b2-f916-436b-b014-0069a9cd25e1.
[2024-06-24 22:56:32Z] Submitting 1 runs, first five are: 6747f34c:8ca8ff5a-4644-4ee3-bd40-e80ba1994e11
[2024-06-24 22:59:09Z] Completing processing run id 8ca8ff5a-4644-4ee3-bd40-e80ba1994e11.

Execution Summary
RunId: blue_music_r6b0v9m0t2
Web View: https://ml.azur

Bad pipe message: %s [b'\xf2\xc4\x8b\xce\xb9\xa83?\x8f\x00-\n\xfb\xf0S\x9e\xed\x12 7\xe1ve\x06\x10\xc3Q\xb9\xe4\xc5\xfe\x90x\xb7\x0e?\xf5s', b'\xd8\xcb8*\xe0r\x93O\xc2\x92L\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00']
Bad pipe message: %s [b'\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07']
Bad pipe message: %s [b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 ]\x9b{\xaba\xc6]\xfc\xa0\x11\xcdY ~\x9c\x15\xe8\x06\x86z\xaa\x9e']
Bad pipe message: %s [b'l7\xe8\xbf3\x8f"]\x80\x868\x8f\x0f\x85|7\x98\xb9\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\

In [ ]:
ml_client.jobs.create_or_update(
        train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ),
        experiment_name="Gini_SVC_N_FEAT_399_2024_06_23_20_00_54_rerun",
    )
ml_client.jobs.stream(pipeline_job.name)